In [5]:
from absbox import API, mkDeal
import pandas as pd 
localAPI = API("http://localhost:8081/", lang="english")
name = "Deal"
dates = {
    "cutoff":"2024-10-31",
    "closing":"2024-10-31",
    "firstPay":"2024-11-30",
    "stated": "2027-10-31",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd"
}

amortising_loan_monthly = ["Mortgage"
            ,{"originBalance": 6922.22
            ,"originRate": ["fix",0.16]
            ,"originTerm": 12
            ,"freq": "Monthly"
            ,"type": "Even"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 6922.22
            ,"currentRate": 0.16
            ,"remainTerm": 12
            ,"status": "current"}]
amortising_loan_quarterly = ["Mortgage"
            ,{"originBalance":  2_966.67    
            ,"originRate": ["fix",0.16]
            ,"originTerm": 4
            ,"freq": "Quarterly"
            ,"type": "Even"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 2_966.67
            ,"currentRate": 0.16
            ,"remainTerm": 4
            ,"status": "current"}]

bullet_loans = ["Loan"
            ,{"originBalance": 12_222.22
            ,"originRate": ["fix",0.16]
            ,"originTerm": 6
            ,"freq": "Monthly"
            ,"type": "i_p"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 12_222.22
            ,"currentRate": 0.16
            ,"remainTerm": 6
            ,"status": "current"}]

j = {"pool_amortizing": {'assets':[amortising_loan_monthly]*100 + [amortising_loan_quarterly]*100},"pool_bullet": {'assets':[bullet_loans]*10}}


accounts = {
    "collections_interest": {"balance": 0},
    "collections_principal_amortizing": {"balance": 0},
    "collections_principal_bullet": {"balance": 0},
    "revolBuyAcc1": {"balance": 0},
    "revolBuyAcc2": {"balance": 0},
    "dealAcc": {"balance": 0},
}
bonds = {
    "A1": {
        "balance": 1_000_000.00,
        "rate": 0.114,
        "originBalance": 30_000_000,
        "originRate": 0.114,
        "startDate": "2024-10-31",
        "rateType": {"floater":[0.049, "SOFR1M",0.065,"MonthEnd"]},
        "maturityDate": "2027-10-31",
        "bondType": {"Sequential":None},
    },
    "EQ": {
        "balance":  111_111.00,
        "rate": 0.0,
        "originBalance": 111_111.00,
        "originRate": 0.0,
        "startDate": "2024-10-31",
        "maturityDate": "2027-10-31",
        "rateType": {"Fixed": 0.00},
        "bondType": {"Equity": None},
    },
}

                                

fees = {"utilisation_fee": {"type": {"annualPctFee": [("excess",("originalBondBalance", "A1"),("bondBalance", "A1"),),0.007],},
                            "feeStart": "2025-01-31","feeEnd": "2026-10-31"},
        "technology_fee": {"type": {"annualPctFee": [("poolBalance",),0.005],},
                            "feeStart": "2025-01-31"},}
waterfall = {
                    "closingDay":[
                        ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",  1_780_000.00)}], 
                        ["fundWith", "revolBuyAcc2", "A1",{"formula": ("const", 220_000.00)}],
                        ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",  197_777.78)}],
                        ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",  24_444.44)}],
                        ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                        ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],

                    ],
                    "Accelerated":[
                                ["calcInt", "A1"],
                                ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",  2_670_000.00)}], 
                                ["fundWith", "revolBuyAcc2", "A1",{"formula": ("const", 330_000.00)}],
                                ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",  296_666.67)}],
                                ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",  36_666.67)}],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance","pool_amortizing"),
                                                    ("poolBalance","pool_bullet"),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.89,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "revolving":[
                                ["calcInt", "A1"],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance","pool_amortizing"),
                                                    ("poolBalance","pool_bullet"),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.9,),)
                                    ,"support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "amortizing":[
                                ["calcInt", "A1"],  # actions if deal is in Amortizing status
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ['transfer','collections_principal_amortizing','dealAcc'],
                                ['transfer','collections_principal_bullet','dealAcc'],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],

                    "cleanUp": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payPrin", "dealAcc", ["EQ"]],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ],

                    "Defaulted": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ["payPrin", "dealAcc", ["EQ"]],],
                            
                }

collects = [
            [["pool_amortizing"],"CollectedPrincipal", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedRecoveries", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedPrepayment", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedInterest", "collections_interest"],
            [["pool_bullet"],"CollectedPrincipal", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedRecoveries", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedPrepayment", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedInterest", "collections_interest"],
        ]
defrate3periods = \
   [ ("ratio",("curPoolCollectionStats", None, _*-1, "Defaults"),("curPoolCollectionStats", None, _*-1, "BegBalance"))
   for _ in range(3) ]

trigger = {
            "AfterCollect": {
                "revolving_trigger": {
                    "condition": ["any", [">", "2026-01-31"]],
                    "effects": ("newStatus", "Revolving"),
                    "status": False,
                    "curable": False,
                },
                "amortization_trigger": {
                    "condition": ["any", [">=", "2026-10-31"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                },
                "DefaultRate_trigger": { #Rolling Default rate trigger
            "condition": ["all", 
                          [">=", "2025-01-31"],
                          [("avgRatio",*defrate3periods),">", 0.06],
            ],
            "effects": ("newStatus", "Defaulted"),
            "status": False,
            "curable": False,
        },},
        "AfterDistribution": {
                "LTV_trigger": {
                    "condition": ["all",
                                  [">", "2024-10-31"],
                                  [
                        (
                            "/",
                            ("bondBalance","A1"),
                            ("sum",
                             ("poolBalance","pool_amortizing"),
                             ("poolBalance","pool_bullet"),
                             ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
                             ),
                        ),
                            ">",
                        0.9,
                    ],],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": True,
                },
            }, 
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": j,
            "accounts": accounts,
            "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": ("PreClosing","Accelerated"),
        }

deal = mkDeal(deal_data)
revol_asset1 = [amortising_loan_monthly]+[amortising_loan_quarterly]
revol_asset2 = bullet_loans

cdr_assump = 0

cdr_assump_1 = 0

revolvingPool1 = (["constant",*revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":cdr_assump_1},None,None,None
                                                                   )
                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":cdr_assump_1},None,None,None)
                   ,None
                   ,None))

amortisingPoolAssump = ("Pool",("Mortgage",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

bulletPoolAssump = ("Pool",("Loan",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

localAPI = API("http://localhost:8081/", lang="english")
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("interest", ("SOFR1M", 0.049), ("EURIBOR1M", 0.0858)),
             ("inspect",("MonthEnd",("bondBalance", "A1")),
              ("MonthEnd",("poolBalance","pool_amortizing")),
              ("MonthEnd",("poolBalance","pool_bullet")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              ,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
                              ,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
                              ),
                              ("call", {"afterDate":"2027-10-31"}),
                    ("revolving"
                       ,{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
                                              ]
         ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.29.13, server:0.29.18

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.29.13, server:0.29.18

Warning Message from server:
Bond EQ is not paid off

In [7]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2024-10-31,2780000.00,0.00,-2000000.00,0.000,0.00,0,0,0.092667,"[<FundWith:A1,1780000.00>, <FundWith:A1,220000..."
2024-11-30,5670000.00,28109.58,-3000000.00,0.114,28109.58,0,0,0.189000,"[[<FundWith:A1,2670000.00>, <FundWith:A1,33000..."
2024-12-31,5637101.38,58093.15,362898.62,0.114,420991.77,0,0,0.187903,"[<PayInt:A1>, <PayPrin:A1>]"
2025-01-31,5017182.14,54579.49,619919.24,0.114,674498.73,0,0,0.167239,"[<PayInt:A1>, <PayPrin:A1>]"
2025-02-28,4376675.84,43876.28,640506.30,0.114,684382.58,0,0,0.145889,"[<PayInt:A1>, <PayPrin:A1>]"
2025-03-31,4012318.68,42375.81,364357.16,0.114,406732.97,0,0,0.133744,"[<PayInt:A1>, <PayPrin:A1>]"
2025-04-30,3032065.97,37594.87,980252.71,0.114,1017847.58,0,0,0.101069,"[<PayInt:A1>, <PayPrin:A1>]"
2025-05-31,2038404.61,29357.04,993661.36,0.114,1023018.40,0,0,0.067947,"[<PayInt:A1>, <PayPrin:A1>]"
2025-06-30,1674833.47,19099.57,363571.14,0.114,382670.71,0,0,0.055828,"[<PayInt:A1>, <PayPrin:A1>]"


In [15]:
r['pool']['flow']['pool_amortizing']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-10-31,988889.00,0.00,0.00,0,0,0,0,0.160000,None,None,0.00,0,0,0,0,0
2024-11-30,2200277.52,173055.32,27687.78,0,0,0,0,0.160000,None,None,173055.32,0,0,0,0,0
2024-12-31,4044699.67,356462.36,54724.94,0,0,0,0,0.160000,None,None,529517.68,0,0,0,0,0
2025-01-31,3465737.83,578961.84,85571.69,0,0,0,0,0.160000,None,None,1108479.52,0,0,0,0,0
2025-02-28,2873466.08,592271.75,82949.25,0,0,0,0,0.160000,None,None,1700751.27,0,0,0,0,0
2025-03-31,2517003.72,356462.36,40466.14,0,0,0,0,0.160000,None,None,2057213.63,0,0,0,0,0
2025-04-30,1938040.88,578962.84,62413.55,0,0,0,0,0.160000,None,None,2636176.47,0,0,0,0,0
2025-05-31,1345769.13,592271.75,59258.07,0,0,0,0,0.160000,None,None,3228448.22,0,0,0,0,0
2025-06-30,989306.77,356462.36,26208.35,0,0,0,0,0.160000,None,None,3584910.58,0,0,0,0,0


In [16]:
r['pool']['flow']['pool_bullet']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2024-10-31,122222.20,0.00,0.00,0,0,0,0,0.16,0.00,0,0,0,0,0
2024-11-30,366666.64,0.00,4821.91,0,0,0,0,0.16,0.00,0,0,0,0,0
2024-12-31,733333.31,0.00,9804.47,0,0,0,0,0.16,0.00,0,0,0,0,0
2025-01-31,733333.31,0.00,9965.20,0,0,0,0,0.16,0.00,0,0,0,0,0
2025-02-28,733333.31,0.00,9161.58,0,0,0,0,0.16,0.00,0,0,0,0,0
2025-03-31,733333.31,0.00,9804.47,0,0,0,0,0.16,0.00,0,0,0,0,0
2025-04-30,366666.67,366666.64,9804.55,0,0,0,0,0.16,366666.64,0,0,0,0,0
2025-05-31,0.00,366666.67,4821.91,0,0,0,0,0.00,733333.31,0,0,0,0,0


In [17]:
r['triggers']['EndDistributionWF']['LTV_trigger']

,status,memo
date,,
2024-11-30,True,<Tag:All:2024-11-30 G 2024-10-31|0.98 G 0.90>
2024-12-31,True,<Tag:All:2024-12-31 G 2024-10-31|1.17 G 0.90>
2025-01-31,True,<Tag:All:2025-01-31 G 2024-10-31|1.19 G 0.90>
2025-02-28,True,<Tag:All:2025-02-28 G 2024-10-31|1.21 G 0.90>
2025-03-31,True,<Tag:All:2025-03-31 G 2024-10-31|1.23 G 0.90>
2025-04-30,True,<Tag:All:2025-04-30 G 2024-10-31|1.31 G 0.90>
2025-05-31,True,<Tag:All:2025-05-31 G 2024-10-31|1.51 G 0.90>
2025-06-30,True,<Tag:All:2025-06-30 G 2024-10-31|1.69 G 0.90>
2025-07-31,True,<Tag:All:2025-07-31 G 2024-10-31|2.61 G 0.90>


In [11]:
r['result']['waterfall']

,Date,Waterfall Location
0,2024-10-31,<OnClosingDay>
1,2024-11-30,<DistributionDay:<DealAccelerated>>
2,2024-12-31,<DistributionDay:<DealDefaulted>>
3,2025-01-31,<DistributionDay:<DealDefaulted>>
4,2025-02-28,<DistributionDay:<DealDefaulted>>
5,2025-03-31,<DistributionDay:<DealDefaulted>>
6,2025-04-30,<DistributionDay:<DealDefaulted>>
7,2025-05-31,<DistributionDay:<DealDefaulted>>
8,2025-06-30,<DistributionDay:<DealDefaulted>>
9,2025-07-31,<DistributionDay:<DealDefaulted>>


In [18]:
r['result']['inspect']['<CurrentBondBalanceOf:A1>']

,<CurrentBondBalanceOf:A1>
Date,
2024-10-31,3000000.00
2024-11-30,6000000.00
2024-12-31,5637101.38
2025-01-31,5017182.14
2025-02-28,4376675.84
2025-03-31,4012318.68
2025-04-30,3032065.97
2025-05-31,2038404.61
2025-06-30,1674833.47


In [13]:
r['result']['inspect']['<CurrentPoolBalance:{\'tag\': \'PoolName\', \'contents\': \'pool_amortizing\'}>']

,"<CurrentPoolBalance:{'tag': 'PoolName', 'contents': 'pool_amortizing'}>"
Date,
2024-10-31,2966666.77
2024-11-30,5344399.61
2024-12-31,4044699.67
2025-01-31,3465737.83
2025-02-28,2873466.08
2025-03-31,2517003.72
2025-04-30,1938040.88
2025-05-31,1345769.13
2025-06-30,989306.77


In [14]:
r['result']['inspect']['<CurrentPoolBalance:{\'tag\': \'PoolName\', \'contents\': \'pool_bullet\'}>']

,"<CurrentPoolBalance:{'tag': 'PoolName', 'contents': 'pool_bullet'}>"
Date,
2024-10-31,366666.64
2024-11-30,733333.31
2024-12-31,733333.31
2025-01-31,733333.31
2025-02-28,733333.31
2025-03-31,733333.31
2025-04-30,366666.67
2025-05-31,0.00
2025-06-30,0.00
